In [1]:
import json
from datasets import load_dataset
from tqdm.auto import tqdm

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rag_dataset = load_dataset("DuyTa/vi_RAG", 'BKAI_RAG')
rag_dataset1 = load_dataset("DuyTa/vi_RAG", 'LegalRAG')
rag_dataset2 = load_dataset("DuyTa/vi_RAG", 'expert')

In [3]:
instances_list = []

In [4]:
def create_data_json(rag_dataset, group_name):
    # Iterate over each instance in the 'train' split
    for instance in rag_dataset['train']:
        # Create a dictionary for the current instance and append it to the list
        instances_list.append({
            "group": group_name,
            "context": ''.join(instance["context"][0]),
            "question": instance["question"],
            "answer": instance["answer"]
        })

In [5]:
create_data_json(rag_dataset, "General")
create_data_json(rag_dataset1, "Legal")

In [6]:
for instance in rag_dataset2['train']:
        # Create a dictionary for the current instance and append it to the list
        instances_list.append({
            "group": "Expert",
            "context": instance["revised_claims"],
            "question": instance["question"],
            "answer": instance["revised_answer"]
        })

In [7]:
for doc in tqdm(instances_list):
    context = doc['context']
    if context is None:
        doc['context'] = 'Ngữ cảnh không được cung cấp'
        print("No context")

100%|██████████████████████████████████| 6089/6089 [00:00<00:00, 2530379.18it/s]

No context
No context
No context


In [8]:
# Save the list of instances to a JSON file
with open('../data/vietnamese_rag/documents.json', 'w', encoding='utf-8') as f:
    json.dump(instances_list, f, ensure_ascii=False, indent=4)

In [9]:
with open('../data/vietnamese_rag/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

I already create documents with id files, but i have modified a little bit in the new documents.json file, so I want to reuse the id from the original documents for consistency

In [10]:
with open('../data/vietnamese_rag/documents-with-ids1.json', 'rt') as f_in:
    documents1 = json.load(f_in)
with open('../data/vietnamese_rag/documents-with-ids2.json', 'rt') as f_in:
    documents2 = json.load(f_in)
with open('../data/vietnamese_rag/documents-with-ids3.json', 'rt') as f_in:
    documents3 = json.load(f_in)
with open('../data/vietnamese_rag/documents-with-ids4.json', 'rt') as f_in:
    documents4 = json.load(f_in)
with open('../data/vietnamese_rag/documents-with-ids5.json', 'rt') as f_in:
    documents5 = json.load(f_in)
original_documents = documents1.copy()
original_documents.extend(documents2)
original_documents.extend(documents3)
original_documents.extend(documents4)
original_documents.extend(documents5)

In [11]:
len(original_documents)

6089

In [12]:
doc_id_list = []
for doc in original_documents:
    doc_id_list.append(doc['id'])

In [13]:
for i in range(len(original_documents)):
    documents[i]['id'] = doc_id_list[i]


In [14]:
documents[0]

{'group': 'General',
 'context': "ngữ cảnh 1: Vượt qua nỗi sợ hãi, Minh Tú xuất sắc vào chung kết Asia's Next Top Model.  Bỏ qua hết những lo lắng về căn bệnh tim, Minh Tú đã hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng và dành chiếc vé vào đêm chung kết Asia’s Next Top Model..  Tối ngày 21/6, tập 12 của chương trình Asia’ Next Top Model mùa thứ 5 đã được lên sóng. 4 thí sinh cùng giám khảo bí mật Xiao Qing bước vào phần thi thử thách đặc biệt là đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng. Các cô gái phải vừa tạo dáng vừa thể hiện được sự mạnh mẽ, quyến rũ riêng.. Mặc dù có bệnh về tim nhưng Minh Tú đã cố gắng hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách này. Thí sinh chiến thắng là Shikin và đại diện Việt Nam cũng không kém cạnh gì khi xuất sắc xếp ở vị trí thứ 2, báo GĐVN đưa tin.. Bước vào thử thách, Minh Tú hét lớn: “Tôi còn nhiều việc chưa làm được đâu” vì sợ sẽ rơi xuống tuy nhiên c

In [15]:
with open('../data/vietnamese_rag/documents-with-ids.json', 'wt') as f_out:
    json.dump(documents , f_out, indent=2)